<a href="https://www.kaggle.com/code/rishabh73/remove-duplicate-images-from-dataset?scriptVersionId=104178958" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### Many times when collecting dataset from real world there is a chance of containg multiple copies (almost copy) of a single image with different names. It can cause an imbalance in dataset and skew the weights of model hence resulting in poor performance. 
### This notebook can be used to remove duplicate images in datasets using image hash library in a fast and efficient manner.

### we will go step by step mentioning the reason for each code block

In [2]:
# I like to use this block to set the  display size and font of jupyter notebooks
import IPython.core.display as di
from IPython.display import display, HTML
from IPython.display import Audio
display(HTML("<style>.container { width:100% !important; }</style>"))
di.display_html("""
('<style>.cell { margin-bottom: 40px !important;}</style>')
""", raw=True)

from IPython.display import display, HTML
display(HTML("<style>.container { height:100% !important; }</style>"))


('')

### Importing the libraries

In [ ]:
# importing some usefull librariies

import glob
from tqdm.auto import tqdm
import random
import pandas as pd
import numpy as np
import os
from itertools import chain
import glob
import shutil

from PIL import Image
import os.path
import imagehash
import multiprocessing


pd.set_option('display.max_colwidth', None)


## Here we first read the dataframe containing the name of all frames, it can be a list also

In [ ]:
df = pd.read_pickle('..../df.pkl')

## Let's import the hash functions based upon which we will creat hash of images.
### what is hash exactly? 
#### you must have heard of hashing in blockchain technology. consider hash as a unique small key to a very big unique lock. hashes are easy to work with because the millions of image as a data can be quite big to handle for computer but hash is just a string. where each unique hash represent a unique image so if  two hashes are same then the images (or any other data) are also same.

### hence our algorithm uses a list of  hashes of given image dataset and then we can easily find duplicate hashes and remove them.

### Now  the question arises how to calculate this Hash??

### we will use a popular library called 'Imagehash' which has multiple algorithms available like average hash, perpetual hash etc.
### They are baed upon the frequency property of an image. high frequencies in image represent details and low frequencies represent  structure.
### you can read about them further here.

#### https://www.hackerfactor.com/blog/index.php?/archives/432-Looks-Like-It.html


In [ ]:
hashing_method = [imagehash.average_hash,imagehash.phash,imagehash.whash,imagehash.dhash] # Here defining a list of hashes we will use

# lets define some useful functions
image_real_dir = 'path to the image folder'

def return_hash(self,row_idx): # here we assume that the column name which contains frame  name is titled 'frame_name'
    '''
    INPUT
    row_idx: int; row number of dataframe containing frame name
    
    OUTPUT
    hashes of image: strings
    '''
    Hash = [] # creating an empty list to store the hash of each image based upon all the hashing method defined above
    img_read = Image.open(os.path.join(img_real_dir,row_idx['frame_name']))
    for func_hash in hashing_method:
        Hash.append(str(func_hash(img_read))) 

    return Hash[0],Hash[1],Hash[2],Hash[3] # returning the hash of an image based upon different algorithms

# we have defined a function which will return the hashes of image and 




tqdm.pandas() # we will use multi processing to engage all cores of cpu
df_dum = df.copy() # keeping a copy of dataframe
rows_iter = (row for _, row in df_dum.iterrows()) # row list 
pool = multiprocessing.Pool() # initiating the multiprocessing pool

# Here we map the function defined above to all the rows(frames) inside dataframe and create four columns titles hash1, hash2, hash3, hash4
# corresponding to the hash algoithms defined above
df_dum['hash1'],df_dum['hash2'],df_dum['hash3'],df_dum['hash4'] = zip(*list(tqdm(pool.imap(return_hash,rows_iter),total=len(df_dum)))) 

pool.close()
pool.join()

# now we will create a set of hash1 column and remove the duplicate rows and then further apply it to remaning hashes

df_dum = df_dum.groupby(['hash1'], as_index=False)[['frame_name','hash2','hash3','hash4']].agg(lambda x: list(set(x)))

df_dum['hash2'] = df_dum['hash2'].apply(lambda x: x[0]) # this returns the remaining hashes in a list containg duplicate hashes hence we are 
# taking the first hash out of list for hash2

df_dum = df_dum.groupby(['hash2'], as_index=False)[['frame_name','hash3','hash4']].agg(lambda x: list(set(list(chain.from_iterable(x)))))
df_dum['hash3'] = df_dum['hash3'].apply(lambda x: x[0])

df_dum = df_dum.groupby(['hash3'], as_index=False)[['frame_name','hash4']].agg(lambda x: list(set(list(chain.from_iterable(x)))))
df_dum['hash4'] = df_dum['hash4'].apply(lambda x: x[0])

df_dum = df_dum.groupby(['hash4'], as_index=False)[['frame_name']].agg(lambda x: list(set(list(chain.from_iterable(x)))))

df_dum['frame_name'] = df_dum['frame_name'].apply(lambda x: x[0])
df_dum.drop(columns='hash4') # removing the last column



In [ ]:
df_dum # it is the final dataframe containg unique imgaes name